In [11]:
import asyncio

from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
import pandas as pd 

async def sub_order(account_address) -> None:
    network = Network.local()
    client = AsyncClient(network)
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]

async def order_list(subaccount_id,market_id) -> None:
    # select network: local, testnet, mainnet
    network = Network.local()
    client = AsyncClient(network)
    skip = 0
    limit = 100
    pagination = PaginationOption(skip=skip, limit=limit)
    orders = await client.fetch_subaccount_orders_list(
        subaccount_id=subaccount_id, market_id=market_id, pagination=pagination
    )
    return orders['orders']

async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    df = df[["orderSide","subaccountId","price","quantity"]]

    return df


market_id = "0x2d7092545081b81ba33bf817b302f9609254f15f4354016631aec3bb39461f99"

#await subfinder(account_address="inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z",market_id="0x2d7092545081b81ba33bf817b302f9609254f15f4354016631aec3bb39461f99")

In [12]:
from functions import marketId_to_ticker

ticker = "PEPE/USDT PERP"
address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

market_id = await marketId_to_ticker(ticker)
await subfinder(account_address=address,market_id=market_id[0])

,orderSide,subaccountId,price,quantity
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000011,15.578,2097000000
1,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000011,15.622,2097000000
